In [8]:
import tensorflow as tf
print(tf.__version__)
print(tf.__file__)
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

2.17.0
/home/martizia/.local/lib/python3.12/site-packages/tensorflow/__init__.py
GPU Available:  []


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import fashion_mnist
import matplotlib.pyplot as plt

2024-07-16 22:35:40.086621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-16 22:35:40.265977: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-16 22:35:40.309043: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-16 22:35:40.630635: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-16 22:35:42.269056: W tensorflow/compiler/tf2

In [3]:
# Завантаження та підготовка даних
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [1]:
# Перетворення зображень у формат RGB (VGG16 очікує 3 канали)
train_images = tf.image.grayscale_to_rgb(tf.expand_dims(train_images, -1))
test_images = tf.image.grayscale_to_rgb(tf.expand_dims(test_images, -1))

NameError: name 'tf' is not defined

In [6]:
# Перетворення в float32 і нормалізація даних
train_images = tf.cast(train_images, tf.float32) / 255.0
test_images = tf.cast(test_images, tf.float32) / 255.0

2024-07-16 17:48:14.957344: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1693440000 exceeds 10% of free system memory.
2024-07-16 17:48:15.734157: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1693440000 exceeds 10% of free system memory.
2024-07-16 17:48:16.869389: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 282240000 exceeds 10% of free system memory.


In [7]:
# Зміна розміру зображень до 32x32 (найменший розмір, який підтримує VGG16)
train_images = tf.image.resize(train_images, (32, 32))
test_images = tf.image.resize(test_images, (32, 32))

ValueError: 'images' must have either 3 or 4 dimensions.

In [25]:
# Завантаження VGG16 без верхніх шарів
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

In [27]:
# Заморожування перших шарів (виділення ознак)
for layer in base_model.layers[:-4]:
    layer.trainable = False

In [29]:
# Створення моделі
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

In [31]:
# Компіляція моделі
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
# Функція для зміни швидкості навчання
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
# Тренування моделі
history = model.fit(train_images, train_labels, epochs=15, 
                    validation_split=0.2,
                    batch_size=64,
                    callbacks=[callback])

Epoch 1/15
458/750 ━━━━━━━━━━━━━━━━━━━━ 7:50 2s/step - accuracy: 0.5241 - loss: 1.3858

In [ ]:
# Розморожування всіх шарів для тонкого налаштування
base_model.trainable = True

In [ ]:
# Перекомпіляція моделі
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Донавчання моделі
history_fine = model.fit(train_images, train_labels, epochs=10, 
                         validation_split=0.2,
                         batch_size=32,
                         callbacks=[callback])

In [ ]:
# Оцінка моделі
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'\nТочність на тестових даних: {test_acc}')

In [ ]:
# Функція для побудови графіків
def plot_learning_curves(history, title_suffix=""):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Графік точності
    ax1.plot(history.history['accuracy'], label='Тренувальна вибірка')
    ax1.plot(history.history['val_accuracy'], label='Валідаційна вибірка')
    ax1.set_title(f'Точність моделі {title_suffix}')
    ax1.set_ylabel('Точність')
    ax1.set_xlabel('Епоха')
    ax1.legend()
    
    # Графік втрат
    ax2.plot(history.history['loss'], label='Тренувальна вибірка')
    ax2.plot(history.history['val_loss'], label='Валідаційна вибірка')
    ax2.set_title(f'Втрати моделі {title_suffix}')
    ax2.set_ylabel('Втрати')
    ax2.set_xlabel('Епоха')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Побудова графіків для першого етапу навчання
plot_learning_curves(history, "(перший етап)")

In [ ]:
# Побудова графіків для етапу тонкого налаштування
plot_learning_curves(history_fine, "(тонке налаштування)")